In [1]:
### lrasmy@Zhilab last revised November 19 2022   ##
###############################################

## Download the required data 

- Make sure to mount your drive using the mount drive icon on the left pane or run the few lines below 



In [ ]:
### Mount your google drive ###
from google.colab import drive
drive.mount('/content/drive')

*   Then download the required MIMIC IV data using the following command 
please make sure you replace --user=<username> with your actual physionet username like --user='jsmith' 
- you will be prompt to enter your password, make sure you click to see a text box and then enter your physionet password.




In [ ]:
!wget -r -N -c -np -i /content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/Data_to_Download.txt -P /content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data --user='jsmith'  --ask-password

## Extract The in-hospital Mortality Data

### Import the required packages


In [17]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
import mimic4_preprocess_util
from mimic4_preprocess_util import *

mimic4_path='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/mimiciv/2.1'
verbose= True
output_path='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/extracted_data'
if not os.path.exists(output_path):
        os.makedirs(output_path)

In [18]:
from importlib import reload
reload(mimic4_preprocess_util)
from mimic4_preprocess_util import *


In [ ]:
stays = read_icustays_table(mimic4_path)
stays=stays.reset_index()
if verbose:
    print('START:', stays.stay_id.unique().shape[0], stays.hadm_id.unique().shape[0],
          stays.subject_id.unique().shape[0])
print(min(stays['subject_id']),max(stays['subject_id']))
stays = clean_stays(stays)
print(min(stays['subject_id']),max(stays['subject_id']))
admits = read_admissions_table(mimic4_path)
stays = merge_on_subject_admission(stays, admits)
print(min(stays['subject_id']),max(stays['subject_id']))

patients = read_patients_table(mimic4_path)

stays = merge_on_subject(stays, patients)

print('latest stay cnts:', stays.stay_id.unique().shape[0], stays.hadm_id.unique().shape[0],
         stays.subject_id.unique().shape[0])
stays = add_age_to_icustays(stays)
stays = add_inunit_mortality_to_icustays(stays)
stays = add_inhospital_mortality_to_icustays(stays)

In [ ]:
stays

In [ ]:
### Quality check
stays[(stays['mortality']==1) & (stays['deathtime'].isna())] [['subject_id', 'hadm_id', 'stay_id', 
       'intime', 'outtime', 'los', 'admittime', 'dischtime', 'deathtime',
       'race', 'gender', 'dod', 'anchor_age', 'anchor_year', 'yob', 'age',
       'mortality_inunit', 'mortality', 'mortality_inhospital']]

In [28]:
diagnosis = pd.read_csv(mimic4_path+'/hosp/diagnoses_icd.csv.gz', compression='gzip', header=0)
admis = pd.read_csv(mimic4_path+'/hosp/admissions.csv.gz', compression='gzip', header=0)
proc= pd.read_csv(mimic4_path+'/hosp/procedures_icd.csv.gz', compression='gzip', header=0)

In [32]:
### Cases for patient level prediction using the last ICU stay
case_pts= stays[['subject_id','intime','dischtime','deathtime']][stays['mortality']== 1]
case_pts['deathtime']=case_pts['deathtime'].fillna(case_pts['dischtime'])
case_pts_index= case_pts.groupby('subject_id')[['intime','deathtime']].max().reset_index()
case_pts_index['tte']=(pd.to_datetime(case_pts_index['deathtime'])-pd.to_datetime(case_pts_index['intime'])).dt.days
case_adm=admis[admis['subject_id'].isin(case_pts['subject_id'].drop_duplicates().tolist())] 
case_adm1=pd.merge(case_pts_index,case_adm, right_on='subject_id', left_on='subject_id') 
case_adm2=case_adm1[pd.to_datetime(case_adm1['admittime']) <= case_adm1['intime']] #24,559 records

In [ ]:
case_adm2

##### Quick Quality check

In [ ]:
#case_adm2[case_adm2['admittime']>case_adm2['dischtime']] ### good
case_adm2[case_adm2['admittime']>case_adm2['deathtime_y']] ### good

In [35]:
### Extract diagnosis data for cases
case_diag=diagnosis[(diagnosis['subject_id'].isin(case_adm2['subject_id'].drop_duplicates().tolist()))&(diagnosis['hadm_id'].isin(case_adm2['hadm_id'].drop_duplicates().tolist()))]
case_diag=case_diag.merge(case_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
case_diag['dischtime']=pd.to_datetime(case_diag['dischtime']).dt.date
case_diag['event_code']='D_ICD'+case_diag['icd_version'].astype('str')+'_'+case_diag['icd_code']
case_diag=case_diag[['subject_id','event_code','dischtime']].drop_duplicates()


In [36]:
### Extract procedure data for cases
case_proc=proc[(proc['subject_id'].isin(case_adm2['subject_id'].drop_duplicates().tolist()))&(proc['hadm_id'].isin(case_adm2['hadm_id'].drop_duplicates().tolist()))]
case_proc=case_proc.merge(case_pts_index.drop_duplicates(),how='left')
case_proc=case_proc[pd.to_datetime(case_proc['chartdate']) <= case_proc['intime']] ### only to keep events occur before the index date
case_proc=case_proc.merge(case_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
case_proc['dischtime']=pd.to_datetime(case_proc['dischtime']).dt.date
case_proc['event_code']='P_ICD'+ case_proc['icd_version'].astype('str')+'_'+case_proc['icd_code']
case_proc=case_proc[['subject_id','event_code','dischtime']].drop_duplicates()

In [37]:
### ctrls for patient level prediction using the last ICU stay

ctrl_pts= stays[['subject_id','intime','dischtime']][~(stays['subject_id'].isin(case_pts['subject_id'].drop_duplicates().tolist()))]
ctrl_pts.columns=['subject_id','intime','last_dischtime']
ctrl_pts_index= ctrl_pts.groupby('subject_id')[['intime','last_dischtime']].max().reset_index()
ctrl_pts_index['tte']=(pd.to_datetime(ctrl_pts_index['last_dischtime'])-pd.to_datetime(ctrl_pts_index['intime'])).dt.days
ctrl_adm=admis[admis['subject_id'].isin(ctrl_pts['subject_id'].drop_duplicates().tolist())] 
ctrl_adm1=pd.merge(ctrl_pts_index,ctrl_adm, right_on='subject_id', left_on='subject_id') 
ctrl_adm2=ctrl_adm1[pd.to_datetime(ctrl_adm1['admittime']) <= ctrl_adm1['intime']] 

### Extract diagnosis data for ctrl

ctrl_diag=diagnosis[(diagnosis['subject_id'].isin(ctrl_adm2['subject_id'].drop_duplicates().tolist()))&(diagnosis['hadm_id'].isin(ctrl_adm2['hadm_id'].drop_duplicates().tolist()))]
ctrl_diag=ctrl_diag.merge(ctrl_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
ctrl_diag['dischtime']=pd.to_datetime(ctrl_diag['dischtime']).dt.date
ctrl_diag['event_code']='D_ICD'+ctrl_diag['icd_version'].astype('str')+'_'+ctrl_diag['icd_code']
ctrl_diag=ctrl_diag[['subject_id','event_code','dischtime']].drop_duplicates()

### Extract procedure data for ctrl
ctrl_proc=proc[(proc['subject_id'].isin(ctrl_adm2['subject_id'].drop_duplicates().tolist()))&(proc['hadm_id'].isin(ctrl_adm2['hadm_id'].drop_duplicates().tolist()))]
ctrl_proc=ctrl_proc.merge(ctrl_pts_index.drop_duplicates(),how='left')
ctrl_proc=ctrl_proc[pd.to_datetime(ctrl_proc['chartdate']) <= ctrl_proc['intime']] ### only to keep events occur before the index date
ctrl_proc=ctrl_proc.merge(ctrl_adm2[['subject_id','hadm_id','admittime','dischtime']].drop_duplicates(),how='left')
ctrl_proc['dischtime']=pd.to_datetime(ctrl_proc['dischtime']).dt.date
ctrl_proc['event_code']='P_ICD'+ ctrl_proc['icd_version'].astype('str')+'_'+ctrl_proc['icd_code']
ctrl_proc=ctrl_proc[['subject_id','event_code','dischtime']].drop_duplicates()


In [38]:
### dump extracted data
pd.concat([case_diag,
           case_proc,
           ctrl_diag,
           ctrl_proc]
          ).dropna().to_csv(output_path+'/Mimic_PT_mortality_data_dp.tsv', sep='\t',index=False)

### dump labels, mortality label and time to death for survival

ctrl_pts_index['mort']=0
case_pts_index['mort']=1

pd.concat([ctrl_pts_index[['subject_id','mort','tte']].drop_duplicates(),
           case_pts_index[['subject_id','mort','tte']].drop_duplicates()
           ]).dropna().to_csv(output_path+'/Mimic_PT_mortality_labels.tsv', sep='\t',index=False)

## Proprocess data into pickled list



In [39]:
sys.path.insert(0,"/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/")
from preprocess_outcomes  import dump_split_process_data

In [40]:
output_proc='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/processed_data'
if not os.path.exists(output_proc):
        os.makedirs(output_proc)

dump_split_process_data(output_path+'/Mimic_PT_mortality_data_dp.tsv', output_path+'/Mimic_PT_mortality_labels.tsv' , 'NA' ,output_proc+'/Mimic_PT_mortality_dp_v1' , 'NA' )

loading data
loaded data for:  50725
loading labels
loaded labels for:  50934  after primary cleaning  50725
Mortality Case counts:  7050
LOS>7 :  14472
new types dictionary
processed 1000 pts
processed 2000 pts
processed 3000 pts
processed 4000 pts
processed 5000 pts
processed 6000 pts
processed 7000 pts
processed 8000 pts
processed 9000 pts
processed 10000 pts
processed 11000 pts
processed 12000 pts
processed 13000 pts
processed 14000 pts
processed 15000 pts
processed 16000 pts
processed 17000 pts
processed 18000 pts
processed 19000 pts
processed 20000 pts
processed 21000 pts
processed 22000 pts
processed 23000 pts
processed 24000 pts
processed 25000 pts
processed 26000 pts
processed 27000 pts
processed 28000 pts
processed 29000 pts
processed 30000 pts
processed 31000 pts
processed 32000 pts
processed 33000 pts
processed 34000 pts
processed 35000 pts
processed 36000 pts
processed 37000 pts
processed 38000 pts
processed 39000 pts
processed 40000 pts
processed 41000 pts
processed 42000

In [41]:
### Extract code descriptions as we will use later

diag_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_diagnoses.csv.gz', compression='gzip', header=0)
diag_desc['event_code']='D_ICD'+diag_desc['icd_version'].astype('str')+'_'+diag_desc['icd_code']
proc_desc= pd.read_csv(mimic4_path+'/hosp/d_icd_procedures.csv.gz', compression='gzip', header=0)
proc_desc['event_code']='P_ICD'+proc_desc['icd_version'].astype('str')+'_'+proc_desc['icd_code']
diag_desc['cat']='Diag'
proc_desc['cat']='Proc'
code_desc=pd.concat([diag_desc[['event_code','cat','long_title']],proc_desc[['event_code','cat','long_title']]])
code_desc.to_csv(output_proc+'/code_desc.tsv',sep='\t',index=False)

## Data in OMOP format (MIMIC IV Demo Data)

> First we need to define our objective: 
  This is a demo data of 100 patients, so we will use all patients till last admission to predict mortality (so note the difference from the previous cohort, is that we don't retrict to icu stays and it is not just in-hospital mortality , i.e we wase our conditions for the sake of the Demo)

Again we will prepare to predict both binary and survival

So we need a label file that include: 'subject_id' , 'mortality_indicator' , 'time from index date to death'

and then extract all patient diagnosis and procedures available till the index date and use the linked visit admission or discharge date to later define the sequence of visits

  So our steps:

  1. Identify the patients label (died or not) --> Death Table
  2. Find out the index date per patient (admission date for last encounter/visit) ----> visit_occurence
  3. Extract diagnosis and procedures, along with the linked visit admission or discharge date





In [ ]:
#!wget -r -N -c -np https://physionet.org/files/mimic-iv-demo-omop/0.9/ -P /content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data 

In [42]:
mimic4_omop_pth='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/mimic-iv-demo-omop/0.9/1_omop_data_csv/'
mo_db_pth='/content/drive/MyDrive/pytorch_ehr/Pytorch_EHR_Tutorial/Data_Prep/data/physionet.org/files/'


### Let's understand the data First



In [43]:
pt_died = pd.read_csv(mimic4_omop_pth+'death.csv', header=0)
pt_died =pt_died[['person_id','death_date']]

In [44]:
pts = pd.read_csv(mimic4_omop_pth+'person.csv', header=0)
pts 


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,3589912774911670296,8507,2095,NaN,NaN,NaN,0,38003563,NaN,NaN,NaN,10009628,M,0,NaN,0,HISPANIC/LATINO,2000001408
1,-3210373572193940939,8507,2079,NaN,NaN,NaN,0,38003563,NaN,NaN,NaN,10011398,M,0,NaN,0,HISPANIC/LATINO,2000001408
2,-775517641933593374,8507,2149,NaN,NaN,NaN,8516,0,NaN,NaN,NaN,10004235,M,0,BLACK/AFRICAN AMERICAN,2000001406,NaN,0
3,-2575767131279873665,8507,2050,NaN,NaN,NaN,8516,0,NaN,NaN,NaN,10024043,M,0,BLACK/AFRICAN AMERICAN,2000001406,NaN,0
4,-8970844422700220177,8507,2114,NaN,NaN,NaN,8527,0,NaN,NaN,NaN,10038933,M,0,WHITE,2000001404,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-7671795861352464589,8532,2052,NaN,NaN,NaN,2000001401,0,NaN,NaN,NaN,10038081,F,0,UNKNOWN,2000001401,NaN,0
96,5734523979606454056,8532,2069,NaN,NaN,NaN,2000001401,0,NaN,NaN,NaN,10036156,F,0,UNKNOWN,2000001401,NaN,0
97,1532249960797525190,8532,2106,NaN,NaN,NaN,2000001405,0,NaN,NaN,NaN,10014078,F,0,UNABLE TO OBTAIN,2000001405,NaN,0
98,5894416985828315484,8532,2055,NaN,NaN,NaN,2000001405,0,NaN,NaN,NaN,10019172,F,0,UNABLE TO OBTAIN,2000001405,NaN,0


In [45]:
vts = pd.read_csv(mimic4_omop_pth+'visit_occurrence.csv', header=0)
vts[['person_id','visit_occurrence_id','visit_start_datetime','visit_end_datetime','admitting_source_value','discharge_to_source_value','preceding_visit_occurrence_id']]

,person_id,visit_occurrence_id,visit_start_datetime,visit_end_datetime,admitting_source_value,discharge_to_source_value,preceding_visit_occurrence_id
0,4783904755296699562,-4406053801395356975,2112-11-06 11:05:00,2112-11-06 11:05:00,NaN,NaN,-3.100296e+18
1,-6225647829918357531,2636026522589494723,2153-10-17 14:23:00,2153-10-17 14:23:00,NaN,NaN,-2.238366e+18
2,7918537411740862407,-8900473297538187901,2130-10-10 16:20:00,2130-10-10 16:20:00,NaN,NaN,-8.600570e+18
3,-6225647829918357531,-8091470952174089624,2155-04-20 09:00:00,2155-04-20 09:00:00,NaN,NaN,4.782862e+18
4,7155255168997124770,8144493519470334167,2147-07-08 08:20:00,2147-07-08 08:20:00,NaN,NaN,-3.275690e+18
...,...,...,...,...,...,...,...
847,4498126063475867818,6557338641781338493,2156-05-11 11:29:00,2156-05-22 14:16:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
848,4498126063475867818,-9026896896997717305,2156-04-30 18:30:00,2156-05-03 16:36:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
849,6128703162302148003,5538617222786573932,2178-07-22 05:41:00,2178-07-25 16:30:00,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,NaN
850,-3908355835367628651,-4177473998334219433,2137-02-24 10:00:00,2137-03-19 15:45:00,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,NaN


In [46]:
diag = pd.read_csv(mimic4_omop_pth+'condition_occurrence.csv', header=0)
diag

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,7000818053728441484,1741351032930224901,196523,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,78791,44824628,NaN,NaN
1,-3514320024333679102,1741351032930224901,436659,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,2809,44828816,NaN,NaN
2,-6297447354934110463,1741351032930224901,435515,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,2761,44822952,NaN,NaN
3,3131132784874295309,1741351032930224901,436096,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,33829,44834629,NaN,NaN
4,2246709778332812674,1741351032930224901,440383,2179-07-24,2179-07-24 18:21:00,2179-07-28,2179-07-28 15:54:00,32821,NaN,NaN,-5779522865065417426,NaN,311,44833421,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16436,5160751222712405682,-1210657672714831684,4064452,2134-01-01,2134-01-01 08:00:00,2134-01-01,2134-01-01 08:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16437,7817722518091634880,-1210657672714831684,4064452,2134-01-01,2134-01-01 12:00:00,2134-01-01,2134-01-01 12:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16438,5631597446422224513,-1210657672714831684,4064452,2134-01-01,2134-01-01 14:00:00,2134-01-01,2134-01-01 14:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN
16439,8817407860694965134,-1210657672714831684,4064452,2134-01-01,2134-01-01 13:00:00,2134-01-01,2134-01-01 13:00:00,32817,NaN,NaN,-3312868479314057914,NaN,AF (Atrial Fibrillation),2000031019,NaN,NaN


#### Some data quality check

In [49]:
diag['condition_source_value']=diag['condition_source_value'].str.strip()
v_diag=diag[diag['condition_source_value'].str.strip().str.lower().str.startswith('v')][['condition_source_value','condition_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes
e_diag=diag[diag['condition_source_value'].str.strip().str.lower().str.startswith(('e0','e8','e9'))][['condition_source_value','condition_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes


In [50]:
diag_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_diagnoses.csv.gz', compression='gzip', header=0)
v_diag=v_diag.merge(diag_desc, how='left' ,left_on='condition_source_value',right_on='icd_code')
v_diag[v_diag['icd_code'].duplicated(keep=False)]

,condition_source_value,condition_type_concept_id,icd_code,icd_version,long_title
3,V462,32821,V462,9.0,"Other dependence on machines, supplemental oxygen"
4,V462,32821,V462,10.0,Person on outside of car injured in collision ...
7,V641,32821,V641,9.0,Surgical or other procedure not carried out be...
8,V641,32821,V641,10.0,Passenger in heavy transport vehicle injured i...
9,V643,32821,V643,9.0,Procedure not carried out for other reasons
10,V643,32821,V643,10.0,Unspecified occupant of heavy transport vehicl...
14,V860,32821,V860,9.0,Estrogen receptor positive status [ER+]
15,V860,32821,V860,10.0,Driver of special all-terrain or other off-roa...
17,V090,32821,V090,9.0,Infection with microorganisms resistant to pen...
18,V090,32821,V090,10.0,Pedestrian injured in nontraffic accident invo...


In [51]:
e_diag=e_diag.merge(diag_desc, how='left' ,left_on='condition_source_value',right_on='icd_code')
e_diag[e_diag['icd_code'].duplicated(keep=False)]

,condition_source_value,condition_type_concept_id,icd_code,icd_version,long_title
12,E8809,32821,E8809,9,Accidental fall on or from other stairs or steps
13,E8809,32821,E8809,10,"Other disorders of plasma-protein metabolism, ..."
23,E8339,32821,E8339,9,Fall on stairs or ladders in water transport i...
24,E8339,32821,E8339,10,Other disorders of phosphorus metabolism
29,E8881,32821,E8881,9,Fall resulting in striking against other object
30,E8881,32821,E8881,10,Metabolic syndrome


In [52]:
pts_toexclude=diag[diag['condition_source_value'].isin(['E8809','E8339','E8881','V090','V860','V462','V641','V643'])]['person_id'].unique().tolist()
len(pts_toexclude)

11

In [53]:
pt_died[pt_died['person_id'].isin(pts_toexclude)] ### I will exclude 4 out of originally 14 cases :), It is a decision point, exclude the whole pt or just those diag records, I prefer to exclude the full patient if it is a small % but for the sake of the demo, I'll only drop those diag and similarly procedures

,person_id,death_date
2,1194579079287927665,2177-03-29
8,-3908355835367628651,2137-09-02
9,4783904755296699562,2116-03-12
13,-9066461348710750663,2185-01-22


In [54]:
proc_o = pd.read_csv(mimic4_omop_pth+'procedure_occurrence.csv', header=0)
proc_o[['person_id','procedure_source_value','procedure_datetime','visit_occurrence_id']]

,person_id,procedure_source_value,procedure_datetime,visit_occurrence_id
0,4783904755296699562,19301,2113-07-18 14:55:00,-433474223361412760
1,7918537411740862407,19303,2129-10-30 13:20:00,7730200099818586525
2,4498126063475867818,27235,2157-07-18 16:49:00,2514722389640694708
3,8090044958540695372,36252,2143-03-19 12:00:00,6734319289435533605
4,5863607150722936210,36830,2142-07-07 16:40:00,-5935156580867756808
...,...,...,...,...
18442,-3908355835367628651,BT1DYZZ,2137-09-02 17:05:00,-3115724380369486208
18443,-3908355835367628651,BT1FYZZ,2137-09-02 17:05:00,-3115724380369486208
18444,-6225647829918357531,D7021ZZ,2155-05-19 18:27:00,-6661057387050813606
18445,-6225647829918357531,D7021ZZ,2155-06-15 16:30:00,-8296465610050985060


In [55]:
proc_desc = pd.read_csv(mimic4_path+'/hosp/d_icd_procedures.csv.gz', compression='gzip', header=0)
#proc_desc['icd_code']=proc_desc['icd_code'].str.strip()
proc_codes=proc_o[['procedure_source_value','procedure_type_concept_id']].drop_duplicates() #32821 are normal diagnosis, 32817 are those derived from ECG/notes
proc_codes['procedure_source_value']=proc_codes['procedure_source_value'].str.strip()
proc_codes=proc_codes.merge(proc_desc, how='left' ,left_on='procedure_source_value',right_on='icd_code')
proc_codes

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title
0,19301,32821,NaN,NaN,NaN
1,19303,32821,NaN,NaN,NaN
2,27235,32821,NaN,NaN,NaN
3,36252,32821,NaN,NaN,NaN
4,36830,32821,NaN,NaN,NaN
...,...,...,...,...,...
306,B54BZZA,32821,B54BZZA,10.0,Ultrasonography of Right Lower Extremity Veins...
307,BT1DYZZ,32821,BT1DYZZ,10.0,"Fluoroscopy of Right Kidney, Ureter and Bladde..."
308,BT1FYZZ,32821,BT1FYZZ,10.0,"Fluoroscopy of Left Kidney, Ureter and Bladder..."
309,D7021ZZ,32821,D7021ZZ,10.0,Beam Radiation of Spleen using Photons 1 - 10 MeV


In [56]:
proc_codes.describe(include='all')

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title
count,311,311.000000,159,159.0,159
unique,311,NaN,159,NaN,159
top,19301,NaN,009600Z,NaN,Drainage of Cerebral Ventricle with Drainage D...
freq,1,NaN,1,NaN,1
mean,NaN,32826.141479,NaN,10.0,NaN
std,NaN,6.195674,NaN,0.0,NaN
min,NaN,32817.000000,NaN,10.0,NaN
25%,NaN,32821.000000,NaN,10.0,NaN
50%,NaN,32821.000000,NaN,10.0,NaN
75%,NaN,32833.000000,NaN,10.0,NaN


In [57]:
proc_desc[proc_desc['icd_code'].str.contains('36830')]

,icd_code,icd_version,long_title


In [59]:
proc[proc['icd_code'].str.contains('683')]['icd_code'].drop_duplicates()

3154      6831
4850      6839
7290      8683
666688    5683
Name: icd_code, dtype: object

**something is wrong mainly because those code are CPT not ICD procedures** :) 

you can just google "procedure icd 36830" ---> again this is a decision point and for the sake of the demo, I will just ignore those, in real practice, I make sure I include them 


And here, where terminology normalization will be helpful ---- https://github.com/ZhiGroup/terminology_representation




In [60]:
proc_codes=proc_codes.dropna()
proc_codes[proc_codes['icd_code'].duplicated(keep=False)] #### so no duplicates issue

,procedure_source_value,procedure_type_concept_id,icd_code,icd_version,long_title


##### Alternate way to prepocess the data using sql for example you can use the below code to create a sqlite database, and you can move to sql from there

In [ ]:
import sqlite3
conn = sqlite3.connect(mo_db_pth+'mimic_omop.db')
pts[['person_id','year_of_birth','gender_source_value','race_source_value','ethnicity_source_value']].to_sql('pts_omop', conn , if_exists='replace', index=False)
vts[['person_id','visit_occurrence_id','visit_start_datetime','visit_end_datetime','admitting_source_value','discharge_to_source_value','preceding_visit_occurrence_id']].to_sql('visits_omop', conn , if_exists='replace', index=False)
pd.read_csv(mimic4_omop_pth+'death.csv', header=0).to_sql('death_omop', conn , if_exists='replace', index=False)
diag[['person_id','visit_occurrence_id','condition_source_value','condition_start_datetime','condition_end_datetime']].to_sql('diag_omop', conn , if_exists='replace', index=False)
proc_o[['person_id','visit_occurrence_id','procedure_source_value','procedure_datetime']].to_sql('proc_omop', conn , if_exists='replace', index=False)
diag_desc.to_sql('diag_desc', conn, if_exists='replace', index=False)
proc_desc.to_sql('diag_desc', conn, if_exists='replace', index=False)
conn.close()
